In [ ]:
from matplotlib import pyplot as plt
import numpy as np
from scipy import ndimage
import abel
from scipy import constants as sc

In [ ]:
commonpath = r'C:\Users\sann7609\Documents\Oxford\ChannelAnalysis_CALA_Sept25_minisforum\interferometry_resolution\data'
IR_10 = np.loadtxt(commonpath+r'\10\phase_maps\80mbar_Bdel560_t0\Interferometry2\AvgPhase.txt')
IR_20 = np.loadtxt(commonpath+r'\20\phase_maps\80mbar_Bdel560_t0\Interferometry2\AvgPhase.txt')
IR_30 = np.loadtxt(commonpath+r'\30\phase_maps\80mbar_Bdel560_t0\Interferometry2\AvgPhase.txt')
IR_40 = np.loadtxt(commonpath+r'\40\phase_maps\80mbar_Bdel560_t0\Interferometry2\AvgPhase.txt')
IR_50 = np.loadtxt(commonpath+r'\50\phase_maps\80mbar_Bdel560_t0\Interferometry2\AvgPhase.txt')
IR_60 = np.loadtxt(commonpath+r'\60\phase_maps\80mbar_Bdel560_t0\Interferometry2\AvgPhase.txt')
IR_70 = np.loadtxt(commonpath+r'\70\phase_maps\80mbar_Bdel560_t0\Interferometry2\AvgPhase.txt')
IR_80 = np.loadtxt(commonpath+r'\80\phase_maps\80mbar_Bdel560_t0\Interferometry2\AvgPhase.txt')
IR_90 = np.loadtxt(commonpath+r'\90\phase_maps\80mbar_Bdel560_t0\Interferometry2\AvgPhase.txt')
IR_100 = np.loadtxt(commonpath+r'\100\phase_maps\80mbar_Bdel560_t0\Interferometry2\AvgPhase.txt')
IR_110 = np.loadtxt(commonpath+r'\110\phase_maps\80mbar_Bdel560_t0\Interferometry2\AvgPhase.txt')
IR_120 = np.loadtxt(commonpath+r'\120\phase_maps\80mbar_Bdel560_t0\Interferometry2\AvgPhase.txt')
IR_130 = np.loadtxt(commonpath+r'\130\phase_maps\80mbar_Bdel560_t0\Interferometry2\AvgPhase.txt')
IR_140 = np.loadtxt(commonpath+r'\140\phase_maps\80mbar_Bdel560_t0\Interferometry2\AvgPhase.txt')
IR_150 = np.loadtxt(commonpath+r'\150\phase_maps\80mbar_Bdel560_t0\Interferometry2\AvgPhase.txt')
IR_160 = np.loadtxt(commonpath+r'\160\phase_maps\80mbar_Bdel560_t0\Interferometry2\AvgPhase.txt')
IR_170 = np.loadtxt(commonpath+r'\170\phase_maps\80mbar_Bdel560_t0\Interferometry2\AvgPhase.txt')
IR_180 = np.loadtxt(commonpath+r'\180\phase_maps\80mbar_Bdel560_t0\Interferometry2\AvgPhase.txt')
IR_190 = np.loadtxt(commonpath+r'\190\phase_maps\80mbar_Bdel560_t0\Interferometry2\AvgPhase.txt')
IR_200 = np.loadtxt(commonpath+r'\200\phase_maps\80mbar_Bdel560_t0\Interferometry2\AvgPhase.txt')

In [ ]:
def pyAbelTransform(PhaseMap, plumeOrientation):
    '''Performs an inverse Abel transform on a (pre-processed, rotated) 2D phase map using the pyAbel library.'''
    # pyAbel requires that nrows is odd:
    if PhaseMap.shape[0] % 2 == 0: #if even number of rows
        PhaseMap = np.append(PhaseMap, np.zeros((1, PhaseMap.shape[1])), axis=0); # Add one more row of zeros at the edge.
    
    # pyAbel assumes symmetry around vertical axis; if the channel/gas plume/whatever else is oriented horizontally, the phase image needs to be transposed first
    if 'horizontal' in plumeOrientation:
        PhaseMap = np.transpose(PhaseMap)
    # now the abel transform can be performed; using pyAbel library: https://pyabel.readthedocs.io/en/latest/readme_link.html   
    AbelInvertedPhase = abel.Transform(PhaseMap, # must be a 2D array
                                    direction='inverse', #transform performed from 2D projection to cylindrical coords --> this is the inverse Abel transform what we want
                                    method='hansenlaw', #abel transform method; I tried a bunch, they should all give the same result if done corretcly!
                                    origin='image_center', #specifies where the center of the image is located
                                    symmetrize_method='average', #method to use for symmetrization; average is safest
                                    symmetry_axis=0, #axis along which the image is symmetric
                                    ).transform;
    # if the channel/gas plume/whatever else was horizontal, transpose the result back to original orientation
    if 'horizontal' in plumeOrientation:
        AbelInvertedPhase = np.transpose(AbelInvertedPhase);
    return AbelInvertedPhase; #return the abel-inverted phase map

In [ ]:
# finding background level
plt.imshow(IR_120[500:600,600:800],cmap='RdBu')

In [ ]:
ycenter = 750
ymi = ycenter-150
yma = ycenter+150
xmi = 300
xma = 600
vmax = 0.6
proj_peak_val = 300
pixsize = 1.07e-6 #pixel size
lambda_laser = 1030e-9 #wavelength of the probe beam
mbar = 80
deg_rotate = 2.0
T_stand = 293.15  # K
ne_full = mbar*(1013.25*100/1000)/(sc.k*T_stand)*1e-6 * 2  # cm^-3 (100% ionization ref for top row), in cm^-3

ne_max = []

#all images
imgs = [IR_10,IR_20,IR_30,IR_40,IR_50,IR_60,IR_70,IR_80,IR_90,IR_100,IR_110,IR_120,IR_120,IR_130,IR_140, IR_150, IR_160, IR_170, IR_180,IR_190, IR_200]

fig, axs = plt.subplots(3,int(len(imgs)),figsize=(10,5),gridspec_kw={'width_ratios': list(np.ones(len(imgs))), 'height_ratios':[1,1,1]})

for i,image in enumerate(imgs):
    ax = axs[0,i]
    rotimage = -ndimage.rotate(image[ymi:yma,xmi:xma],deg_rotate,reshape=False)
    rotimage += np.mean(image[500:600,600:800])

    extent = [0, len(rotimage)*pixsize*1e6, -len(rotimage)*pixsize*1e6/2, len(rotimage)*pixsize*1e6/2]
    ax.imshow(-rotimage, cmap='RdBu',vmin=-vmax, vmax=vmax,extent=extent, origin='lower')

    proj = np.mean(rotimage, axis=1)
    xproj = np.linspace(-len(proj)*pixsize*1e6/2, len(proj)*pixsize*1e6/2, len(proj))
    ax.plot(proj/np.max(np.abs(proj))*proj_peak_val,xproj,color='black')
    ax.axhline(0,color='black',linestyle='dashed',lw=1)

    if i!=0:
        ax.set_yticklabels([])

    const = -sc.e**2/(8*np.pi**2*sc.epsilon_0*sc.m_e*sc.c**2)*lambda_laser**2
    ne_2D = pyAbelTransform(PhaseMap=rotimage*lambda_laser/(2*np.pi*pixsize*1e-6), plumeOrientation='horizontal')/const*1e-6  # in cm^-3
    axs[1,i].imshow(ne_2D/1e6,cmap='RdBu',extent=extent,vmin=-ne_full, vmax=ne_full, origin='lower')

    proj_abel = -np.mean(ne_2D, axis=1)
    xabel = np.linspace(-len(proj)*pixsize*1e6/2, len(proj)*pixsize*1e6/2, len(proj_abel))
    axs[2,i].plot(xabel,proj_abel*1e-6,color='black')
    axs[2,i].set_xlim([-200,200])
    axs[2,i].set_ylim([0,ne_full*1.1])
    axs[2,i].axhline(ne_full, color='red', linestyle='--')

    ne_max.append(np.max(proj_abel)*1e-6)

axs[0,0].set_ylabel('y (µm)')

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.patches as patches
from PIL import Image, ImageDraw, ImageEnhance
from scipy import ndimage, constants as sc
import imageio

# --- User parameters / constants ---
ycenter = 750
ymi = ycenter-110
yma = ycenter+110
xmi = 300
xma = xmi+2*110+1
vmax = 0.6
proj_peak_val = 130
pixsize = 1.07e-6 # pixel size in meters
lambda_laser = 1030e-9 # wavelength of the probe beam
mbar = 80
deg_rotate = 2.0
T_stand = 293.15  # K
ne_full = mbar*(1013.25*100/1000)/(sc.k*T_stand)*1e-6 * 2  # cm^-3 (100% ionization ref for top row)

# --- Your data placeholders (these must be defined earlier in your workspace) ---
# imgs_show = [IR_20,IR_50,IR_80,IR_120,IR_160,IR_200]
# FFT_heights = [20,50,80,120,150,200]
# ne_max must also exist (used in bottom-right plot)
# IR_150 must exist (used for showmap)
# raw_interf_path image will be read later

# --- Example: assume these are defined; if not, the code will fail below when executed ---
imgs_show = [IR_20, IR_50, IR_80, IR_120, IR_160, IR_200]
FFT_heights = [20, 50, 80, 120, 150, 200]

# -------------------------------------------------------------------------------
# Figure & gridspec layout
fig = plt.figure(figsize=(14, 8))
gs = gridspec.GridSpec(
    3, int(len(imgs_show)),
    height_ratios=[1, 1, 1.55],  # adjust relative heights
    hspace=0.25, wspace=0.05
)

axs = np.empty((2, len(imgs_show)), dtype=object)
for row in range(2):
    for col in range(len(imgs_show)):
        axs[row, col] = fig.add_subplot(gs[row, col])

# --- Main top 2 rows plotting (phase maps and ne) ---
for i,image, FFT_height in zip(range(len(imgs_show)), imgs_show, FFT_heights):
    ax = axs[0,i]
    rotimage = -ndimage.rotate(image[ymi:yma, xmi:xma], deg_rotate, reshape=False)
    rotimage += np.mean(image[500:600, 600:800])

    extent = [0, len(rotimage[0])*pixsize*1e6, -len(rotimage)*pixsize*1e6/2, len(rotimage)*pixsize*1e6/2]
    im0 = ax.imshow(-rotimage, cmap='Blues_r', vmin=-vmax, vmax=0, extent=extent, origin='lower')
    ax.set_xlim(0, len(rotimage[0])*pixsize*1e6)

    proj = np.mean(rotimage, axis=1)
    proj -= proj[0]
    xproj = np.linspace(-len(proj)*pixsize*1e6/2, len(proj)*pixsize*1e6/2, len(proj))
    ax.plot(-proj/0.5*proj_peak_val + len(rotimage[1])*pixsize*1e6, xproj, color='black')
    ax.axhline(0, color='black', linestyle='dotted', lw=1)
    ax.xaxis.tick_top()
    ax.set_xlabel('x (µm)',size=12)
    ax.xaxis.set_label_position('top')
    ax.text(s=f'{2*FFT_height} pix', x=70, y=-160, color='black',size=13)

    if i != 0:
        ax.set_yticklabels([])
        axs[1, i].set_yticklabels([])

    const = -sc.e**2/(8*np.pi**2*sc.epsilon_0*sc.m_e*sc.c**2)*lambda_laser**2
    ne_2D_T = pyAbelTransform(PhaseMap=rotimage*lambda_laser/(2*np.pi*pixsize*1e-6), plumeOrientation='horizontal')/const*1e-6
    ne_2D = np.transpose(ne_2D_T)
    im1 = axs[1, i].imshow(-ne_2D/1e24, cmap='Reds',
                           extent=[extent[2], extent[3], extent[0], extent[1]],
                           vmin=0, vmax=1.5*ne_full*1e-18, origin='lower')

    proj_abel = -np.mean(ne_2D, axis=0)
    peak_zloc = 80
    xabel = np.linspace(-len(proj)*pixsize*1e6/2, len(proj)*pixsize*1e6/2, len(proj_abel))
    ne_frac = np.array(proj_abel)*1e-6/ne_full*peak_zloc

    axs[1, i].plot(xabel, ne_frac, color='black')
    axs[1, i].hlines(y=peak_zloc, xmin=0, xmax=len(proj)*pixsize*1e6/2, color='red', ls='dashed')
    axs[1, i].set_ylim([0, len(rotimage[0])*pixsize*1e6])

for letter in ['a', 'b', 'c', 'd', 'e', 'f']:
    axs[0, ord(letter) - ord('a')].text(s=letter+')', x=15, y=75, color='black', size=15)
for letter in ['g', 'h', 'i', 'j', 'k', 'l']:
    axs[1, ord(letter) - ord('g')].text(s=letter+')', x=-100, y=200, color='black', size=15)

axs[0,0].set_ylabel('y (µm)')
axs[0,0].text(s='projection', x=100, y=80, color='black', size=10)
axs[1,0].text(s='   100%\nionization', x=1, y=peak_zloc*1.1, color='red', size=10)
axs[1,0].set_ylabel('z (µm)')
axs[1,0].text(s='projection', x=5, y=40, color='black', size=10)

cbar0 = fig.colorbar(im0, ax=axs[0, :], orientation='vertical', fraction=0.02, pad=0.04)
cbar0.set_label('Phase [rad]',size=13)

cbar1 = fig.colorbar(im1, ax=axs[1, :], orientation='vertical', fraction=0.02, pad=0.04)
cbar1.set_label(r'$n_e$ [10$^18$ cm$^{-3}$]',size=13)

# -------------------------------------------------------------------------------
# Bottom row: three panels (left, middle (FFT), right (metrics))
ax_bottom_left = fig.add_subplot(gs[2:, 0:2])
ax_bottom_middle = fig.add_subplot(gs[2:, 2:4])
# Instead of a single ax_bottom_right we'll subdivide that GS cell into two stacked axes:
gs_right = gs[2:, 4:].subgridspec(2, 1, height_ratios=[1, 1], hspace=0.0)
ax_bottom_right_top = fig.add_subplot(gs_right[0, 0])
ax_bottom_right_bot = fig.add_subplot(gs_right[1, 0], sharex=ax_bottom_right_top)

# --- Parameters for FFT images (tweak paths / crop) ---
IR_fftmap = r'C:\Users\sann7609\Documents\Oxford\ChannelAnalysis_CALA_Sept25_minisforum\interferometry_resolution\data\150\phase_maps\80mbar_Bdel560_t0\Interferometry2\2DFFT_sig.png'
IR_rects = [
    {"center": (725, 456), "size": (20, 40)},
    {"center": (725, 456), "size": (50, 100)},
    {"center": (725, 456), "size": (80, 200)},
    {"center": (725, 456), "size": (150, 300)},
    {"center": (725, 456), "size": (200, 400)},
]

colors = ['red', 'blue', 'lime', 'magenta']
outline_width = 2
crop_edge_y = 280  # visible window crop
crop_edge_x = 350

def load_image(path):
    if not os.path.exists(path):
        raise FileNotFoundError(f"File not found: {path}")
    return Image.open(path).convert("RGBA")

def rect_bbox_from_center(center, size):
    cx, cy = center
    w, h = size
    x0 = cx - w/2
    y0 = cy - h/2
    x1 = cx + w/2
    y1 = cy + h/2
    return [int(round(x0)), int(round(y0)), int(round(x1)), int(round(y1))]

def draw_rectangles_on_image(img_pil, rects, colors_list, outline_width=2):
    img = img_pil.copy()
    draw = ImageDraw.Draw(img)
    n_colors = len(colors_list)
    for i, r in enumerate(rects):
        bbox = rect_bbox_from_center(r["center"], r["size"])
        color = colors_list[i % n_colors]
        for offset in range(outline_width):
            draw.rectangle([bbox[0]-offset, bbox[1]-offset, bbox[2]+offset, bbox[3]+offset], outline=color)
    return img

# --- Load FFT image and draw rectangles ---
try:
    img_ir = load_image(IR_fftmap)
except FileNotFoundError:
    img_ir = None
    print(f"Warning: IR FFT not found at {IR_fftmap}")

img_ir_with_rects = None
if img_ir is not None:
    img_ir_with_rects = draw_rectangles_on_image(img_ir, IR_rects, colors, outline_width=outline_width)

# --- Display FFT in bottom middle and add k-space labels (CORRECTED) ---
if img_ir_with_rects is not None:
    arr_ir = np.asarray(img_ir_with_rects)
    arr_ir_inv = arr_ir.copy()
    arr_ir_inv[..., :3] = 255 - arr_ir_inv[..., :3]  # invert RGB only
    arr_bright = arr_ir_inv.copy().astype(float)
    arr_bright[..., :3] = 255 * (arr_bright[..., :3] / 255) ** 1
    arr_bright = np.clip(arr_bright, 0, 255).astype(np.uint8)

    ax_bottom_middle.imshow(arr_bright, origin='upper')

    # image pixel dimensions
    ny, nx = arr_bright.shape[0], arr_bright.shape[1]
    h_ir, w_ir = ny, nx

    # crop the visible region (data coordinates = pixel coords for the image)
    ax_bottom_middle.set_xlim([crop_edge_x, w_ir - crop_edge_x])
    ax_bottom_middle.set_ylim([h_ir - crop_edge_y, crop_edge_y])  # origin='upper' so invert

    # ---- CORRECT K-AXIS MAPPING ----
    # per-pixel k step (cycles per µm)
    delta_k = 1.0 / (nx * pixsize * 1e6)   # cycles / µm per pixel
    # center pixel indices (zero frequency location)
    pcx = nx / 2.0
    pcy = ny / 2.0

    # displayed pixel-range (in pixel coordinates)
    x0, x1 = crop_edge_x, w_ir - crop_edge_x
    y0, y1 = crop_edge_y, h_ir - crop_edge_y

    # choose nticks ticks across the displayed region (in pixel coordinates)
    nticks = 5
    xticks_pix = np.linspace(x0, x1, nticks)
    yticks_pix = np.linspace(h_ir - y0, h_ir - y1, nticks)  # convert because ylim was inverted

    # convert pixel positions to k (cycles per µm)
    kx_ticks = (xticks_pix - pcx) * delta_k
    ky_ticks = (pcy - (yticks_pix)) * delta_k   # sign flip to have ky positive upward (adjust as needed)

    # set ticks (pixel positions) and formatted labels
    ax_bottom_middle.set_xticks(xticks_pix)
    ax_bottom_middle.set_xticklabels([f"{v:.2f}" for v in kx_ticks])
    ax_bottom_middle.set_yticks(yticks_pix)
    ax_bottom_middle.set_yticklabels([f"{v:.2f}" for v in ky_ticks])

    ax_bottom_middle.set_xlabel(r"$k_x$ (cycles µm$^{-1}$)",size=12)
    ax_bottom_middle.set_ylabel(r"$k_y$ (cycles µm$^{-1}$)",size=12)

ax_bottom_middle.text(s='      FFT of\n     the raw \ninterferogram\n   (cropped)', x=crop_edge_x + 10, y=crop_edge_y + 170, color='white', size=13)

# -------------------------------------------------------------------------------
# Left bottom: show phase map and overlay raw interferogram with transparency
showmap = -IR_150 + np.mean(IR_150[0:50, 0:50])
ax_bottom_left.imshow(showmap, cmap='RdBu', extent=[0, len(showmap[0])*pixsize*1e6, 0, len(showmap)*pixsize*1e6], origin='lower')
ax_bottom_left.set_xlabel('x (µm)',size=12)
ax_bottom_left.set_ylabel('y (µm)',size=12)
ax_bottom_left.text(s='  Phase shift map\n(250 pix FFT window)', x=40, y=50, color='black', size=13)

raw_interf_path = r'interferometry2_shot.tiff'
raw = np.flip(imageio.v2.imread(raw_interf_path), 0)   # shape: (rows, cols)
if raw.ndim == 3:
    raw = raw[..., 0]

alpha_raw = np.zeros_like(raw, dtype=float)
alpha_raw[:int(0.55*raw.shape[0]), int(0.45*raw.shape[1]):] = 1  # right/top area opaque, rest transparent
# Plot raw with per-pixel alpha (imshow accepts alpha array)
ax_bottom_left.imshow(raw, origin='upper', cmap='gray', alpha=alpha_raw,
                      extent=[0, len(showmap[0])*pixsize*1e6, 0, len(showmap)*pixsize*1e6])

# Draw ROI & background rectangles
rect = patches.Rectangle(
    (xmi*pixsize*1e6, ymi*pixsize*1e6),
    (xma - xmi)*pixsize*1e6,
    (yma - ymi)*pixsize*1e6,
    linewidth=1.2, edgecolor='black', facecolor='none', linestyle='dashed'
)
ax_bottom_left.add_patch(rect)
ax_bottom_left.text(s='ROI', x=(xmi+10)*pixsize*1e6, y=(ymi+20)*pixsize*1e6, color='black', size=12)

rect2 = patches.Rectangle(
    (-250 + xmi*pixsize*1e6, -200 + ymi*pixsize*1e6),
    2*(xma - xmi)*pixsize*1e6,
    0.5*(yma - ymi)*pixsize*1e6,
    linewidth=1.2, edgecolor='black', facecolor='none', linestyle='dashed'
)
ax_bottom_left.add_patch(rect2)
ax_bottom_left.text(s='background', x=-250 + (xmi+10)*pixsize*1e6, y=-200 + (ymi+20)*pixsize*1e6, color='black', size=11)
ax_bottom_left.text(s='       Raw\ninterferogram', x=700, y=900, color='white', size=12)

# -------------------------------------------------------------------------------
# Bottom-right: two stacked subplots (share x-axis)
D_arr = 2*np.linspace(10,200,len(ne_max))
D_arr_fine = 2*np.linspace(10,200,1000)
res_FFT = np.shape(IR_150)[0]*pixsize/D_arr_fine*1e6 #FFT resolution

ne_max = np.array(ne_max)

# top panel: apparent on-axis ionization [%]
ax_bottom_right_top.plot(D_arr, 100*ne_max/ne_full, marker='.', lw=1.2, color='black')
ax_bottom_right_top.axhline(100, color='red', linestyle='--')
ax_bottom_right_top.text(s='100 % ionization', x=32, y=110, color='red',size=12)
ax_bottom_right_top.text(s='o)', x=5, y=112, color='black',size=13)
ax_bottom_right_top.set_xlim(0,410)
ax_bottom_right_top.set_ylim(0,140)
ax_bottom_right_top.set_ylabel('[%]',size=12)
ax_bottom_right_top.yaxis.set_label_position('right')
ax_bottom_right_top.yaxis.tick_right()

ax_bottom_right_top.vlines(x=240, ymin=0, ymax=100, color='blue', linestyle='dotted', lw=1.5)
ax_bottom_right_bot.vlines(x=240, ymin=0, ymax=100, color='blue', linestyle='dotted', lw=1.5)
ax_bottom_right_top.text(s='optimum', x=245, y=60, color='blue',size=12)
ax_bottom_right_top.text(s='Apparent on-axis\n      ionization', x=65, y=10, color='black',size=11)

# bottom panel: FFT resolution (shared x-axis)
ax_bottom_right_bot.plot(D_arr_fine, res_FFT, color='green', label='FFT resolution')
ax_bottom_right_bot.set_ylim(0,25)
ax_bottom_right_bot.set_xlim(0,410)
ax_bottom_right_bot.set_xlabel('FFT window size (pixels)')
ax_bottom_right_bot.text(s='p)', x=5, y=20, color='black',size=13)
ax_bottom_right_bot.set_ylabel('[µm]',size=12)
ax_bottom_right_bot.yaxis.set_label_position('right')
ax_bottom_right_bot.yaxis.tick_right()
ax_bottom_right_bot.text(s='  spatial\nresolution\n  along y', x=107, y=13, color='green',size=11)

ax_bottom_right_top.grid(True, which='both', linestyle='--', linewidth=0.5,alpha=0.5)
ax_bottom_right_bot.grid(True, which='both', linestyle='--', linewidth=0.5,alpha=0.5)

theta_base_deg = 8
eta_axi = 1.45
theta_base_rad = theta_base_deg*np.pi/180
alpha_axi = np.arcsin(eta_axi*np.sin(theta_base_rad)) - theta_base_rad
r00 = 0.194*1.030/np.sin(alpha_axi) #HWHM of the axicon peak
r0 = 0.382*1.030/np.sin(alpha_axi) #radius of the first zero of the axicon peak
r1 = 0.610*1.030/np.sin(alpha_axi) #radius of the first zero of the axicon peak

ax_bottom_right_bot.axhline(2*r00, color='black', linestyle='dotted')
ax_bottom_right_bot.text(s='axicon focus size', x=10, y=1*r00-0.5, color='black',size=10)


# hide x tick labels on top plot to avoid duplication
plt.setp(ax_bottom_right_top.get_xticklabels(), visible=False)

# Tight layout and show
plt.tight_layout()


axs[0,0].text(s='      ROI\nphase shift', x=40, y=-100, color='black', size=13)
axs[1,0].text(s='Abel-inverted\nelectron dens.', x=-58, y=170, color='black', size=11)

ax_bottom_middle.text(s='400 pix',x=660, y=690, color='cyan', size=11)
ax_bottom_middle.text(s='320 pix',x=660, y=640, color='lime', size=11)
ax_bottom_middle.text(s='200 pix',x=660, y=590, color='magenta', size=11)

ax_bottom_left.text(s='m)',x=40, y=970, color='black', size=14)
ax_bottom_middle.text(s='n)',x=360, y=330, color='black', size=14)

plt.savefig('Fig_interferometry_resolution_analysis.png', dpi=300, bbox_inches='tight')

In [ ]:
D_arr = 2*np.linspace(10,200,len(ne_max))
D_arr_fine = 2*np.linspace(10,200,1000)
res_FFT = np.shape(IR_150)[0]*pixsize/D_arr_fine*1e6 #FFT resolution

fix, ax = plt.subplots()

ax.plot(D_arr,ne_max, marker='o')
ax.axhline(ne_full, color='red', linestyle='--')
ax.text(s='100 % ionization', x=27, y=ne_full*1.03, color='red')
ax.set_xlim(0,410)
ax.set_ylim(0,ne_full*1.2)

ax1 = ax.twinx()
ax1.plot(D_arr_fine, res_FFT,color='black',label='FFT resolution')
ax1.axhline(y=3.14,color='red',lw=1,ls='dotted')
ax1.set_ylim(0,80)

ax.set_title('apparent peak ionization and FFT resolution\nvs FFT window size')

In [ ]:
# Jupyter cell: draw rectangles on two FFT PNGs and show side-by-side
# Requirements: pillow, matplotlib, numpy
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
import numpy as np
import os

# ---------------- USER INPUT ----------------
# Replace these with your local paths (or uploaded notebook paths)
IR_fftmap = r'C:\Users\sann7609\Documents\Oxford\ChannelAnalysis_CALA_Sept25_minisforum\interferometry_resolution\data\150\phase_maps\80mbar_Bdel560_t0\Interferometry2\2DFFT_sig.png'
Green_fftmap = r'C:\Users\sann7609\Documents\Oxford\ChannelAnalysis_CALA_Sept25_minisforum\interferometry_resolution\data\150\phase_maps\80mbar_Bdel560_t0\Interferometry1\2DFFT_sig.png'

# Rectangles: lists of dicts for each image.
# Each dict: "center": (x, y)  -- integer pixel coords, origin (0,0) top-left
#              "size":   (width, height) -- integer pixels
# Note: do NOT include color here. Colors are defined in 'colors' below and applied by index.
IR_rects = [
    {"center": (725, 456), "size": (20, 40)},
    {"center": (725, 456), "size": (50, 100)},
    {"center": (725, 456), "size": (100, 200)},
    {"center": (725, 456), "size": (150, 300)},
    {"center": (725, 456), "size": (200, 400)},
    # add more...
]

Green_rects = [
    {"center": (737+75, 490+150), "size": (20, 40)},
    {"center": (737+75, 490+150), "size": (50, 100)},
    {"center": (737+75, 490+150), "size": (100, 200)},
    {"center": (737+75, 490+150), "size": (150, 300)},
    {"center": (737+75, 490+150), "size": (200, 400)},
    # can have a different number of rects than IR_rects
]

# Global color list: rectangle index 0 -> colors[0] on both images, index 1 -> colors[1], etc.
# colors may be color names ('red'), hex '#RRGGBB', or RGB tuples (R,G,B).
# If there are more rectangles than colors, the colors list will cycle.
colors = ['red', 'blue', 'lime', 'magenta']  # edit as desired

# Drawing / output parameters
outline_width = 2        # stroke width in pixels
save_combined = True
out_combined_path = 'fft_with_rects_combined.png'
figsize = (12, 6)        # matplotlib figure size
# --------------------------------------------

def load_image(path):
    """Load image and convert to RGBA. Raises FileNotFoundError if missing."""
    if not os.path.exists(path):
        raise FileNotFoundError(f"File not found: {path}")
    return Image.open(path).convert("RGBA")

def rect_bbox_from_center(center, size):
    """Return integer bbox [x0,y0,x1,y1] given center (x,y) and size (w,h)."""
    cx, cy = center
    w, h = size
    x0 = cx - w/2
    y0 = cy - h/2
    x1 = cx + w/2
    y1 = cy + h/2
    # round to nearest integer for pixel coordinates
    return [int(round(x0)), int(round(y0)), int(round(x1)), int(round(y1))]

def draw_rectangles_on_image(img_pil, rects, colors_list, outline_width=2):
    """
    Draw rectangles on a copy of img_pil.
    rects: list of {"center":(x,y), "size":(w,h)}
    colors_list: list of color specifications; index into this list selects color for each rectangle.
    outline_width: thickness in pixels.
    Returns the modified PIL.Image (a copy).
    """
    img = img_pil.copy()
    draw = ImageDraw.Draw(img)
    n_colors = len(colors_list)
    for i, r in enumerate(rects):
        bbox = rect_bbox_from_center(r["center"], r["size"])
        color = colors_list[i % n_colors]  # cycle if not enough colors
        # For thicker outlines, draw multiple rectangles outward
        for offset in range(outline_width):
            draw.rectangle([bbox[0]-offset, bbox[1]-offset, bbox[2]+offset, bbox[3]+offset], outline=color)
    return img

def make_combined_plot(img1, img2, rects1, rects2, colors_list, save_path=None, figsize=(12,6), outline_width=2):
    """Draw rectangles on both images (colors aligned by index) and display/save side-by-side figure."""
    # draw on copies
    img1_d = draw_rectangles_on_image(img1, rects1, colors_list, outline_width=outline_width)
    img2_d = draw_rectangles_on_image(img2, rects2, colors_list, outline_width=outline_width)

    # Create side-by-side plot
    fig, axes = plt.subplots(1, 2, figsize=figsize)
    axes[0].imshow(np.asarray(img1_d))
    axes[0].axis('off')
    axes[0].set_title('IR FFT')
    crop_edge = 200
    axes[0].set_xlim([crop_edge,1282-crop_edge])
    axes[0].set_ylim([crop_edge,1026-crop_edge])

    axes[1].imshow(np.asarray(img2_d))
    axes[1].axis('off')
    axes[1].set_title('Green FFT')
    axes[1].set_xlim([crop_edge,1282-crop_edge])
    axes[1].set_ylim([crop_edge,1026-crop_edge])

    plt.tight_layout()
    if save_path:
        fig.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"Saved combined image to: {save_path}")
    plt.show()
    return img1_d, img2_d  # return PIL images with rectangles if caller wants to save separately

# ----- Run -----
if __name__ == "__main__" or True:
    # Load images
    try:
        img_ir = load_image(IR_fftmap)
        img_green = load_image(Green_fftmap)
    except FileNotFoundError as e:
        # Helpful error and stop if file not found
        raise

    # Create and show the combined plot (and save if requested)
    img_ir_with_rects, img_green_with_rects = make_combined_plot(
        img_ir, img_green, IR_rects, Green_rects, colors,
        save_path=out_combined_path if save_combined else None,
        figsize=figsize, outline_width=outline_width
    )

    # Optional: save individual images with rectangles
    # img_ir_with_rects.save('IR_with_rects.png')
    # img_green_with_rects.save('Green_with_rects.png')

In [ ]:
theta_base_deg = 8
eta_axi = 1.45
theta_base_rad = theta_base_deg*np.pi/180
alpha_axi = np.arcsin(eta_axi*np.sin(theta_base_rad)) - theta_base_rad
r00 = 0.194*1.030/np.sin(alpha_axi) #HWHM of the axicon peak
r0 = 0.382*1.030/np.sin(alpha_axi) #radius of the first zero of the axicon peak
r1 = 0.610*1.030/np.sin(alpha_axi) #radius of the first zero of the axicon peak

print(2*r0)

In [ ]:
import imageio
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import re
from scipy import ndimage
import math
import imageio.v2 as imageio

tiff_file = "Axicon_240315_8degnoplug_72mm.tif"

center_coords=[550,897]
size = 150
pixel_size = 0.7 #um/pixel
print(int(center_coords[0]-size/2))
x1 = int(center_coords[0]-size/2)
x2 = int(center_coords[0]+size/2)
y1 = int(center_coords[1]-size/2)
y2 = int(center_coords[1]+size/2)
bg = imageio.imread(tiff_file)[0:50,0:50]
image = imageio.imread(tiff_file)[x1:x2,y1:y2]-np.mean(bg)
image = image/np.amax(image)
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)

            # Crop the image around the largest circle
im = ax.imshow(image, cmap='BuPu',vmin=0,vmax=np.amax(image),extent=[-1/2*size*pixel_size,1/2*size*pixel_size,-1/2*size*pixel_size,1/2*size*pixel_size])
#ax.set_xlim(750,1050)
#ax.set_ylim(450,650)
#change_ticks_axicon(data=image,pixsize=1.5,axis=ax,change_X=True,change_Y=True)

#ax.set_title('8 deg axicon focus')

ver_lineout = image[int(size/2)]/np.max(image[int(size/2)])
hor_lineout = image[:,int(size/2)]/np.max(image[:,int(size/2)])
x_lineout = np.linspace(-size/2*pixel_size,size/2*pixel_size,size)
ax.plot(x_lineout,30*ver_lineout-50,color='black')
ax.plot(30*hor_lineout-50,x_lineout,color='black')

ax.set_xlabel(r'X [$\mu$m]',size=15)
ax.set_ylabel(r'Y [$\mu$m]',size=15)

cbar = fig.colorbar(im, orientation='vertical')
cbar.ax.set_ylabel('intensity [arb.units]',size=15)

theta_base_deg = 8
eta_axi=1.45
theta_base_rad = theta_base_deg*np.pi/180
alpha_axi = np.arcsin(eta_axi*np.sin(theta_base_rad))-theta_base_rad #approach angle of the axicon
#ax.vlines(r1,ymin=-50,ymax=-20,color='black',ls='dotted',lw=1)
ax.vlines(r0*0.8,ymin=-50,ymax=-20,color='black',ls='dotted',lw=1)
ax.vlines(r1*0.9,ymin=-50,ymax=-20,color='black',ls='dotted',lw=1)

ax.text(x=r0*0.6,y=-19,s=r'$r_0$')
ax.text(x=r1*0.8,y=-19,s=r'$r_1$')